# Import libs und Daten

In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
# XML-Lib laden
import xml.etree.ElementTree as ET

In [2]:
url_xml = "./test_daten/F.210906.132800.WK.xml"
root = ET.parse(url_xml).getroot()

# Herauslesen Daten Dataframe long

In [3]:
#HELPER
# Ordnet die Beschreibung des Tags <Gruppenergebnis> einer Klarbzeichung zu
# diese Beschreibung ist unter dem Attribut Position abgespeichert

# Check ob sich in einem String eine Nummer befindet
def has_numbers(inputString):
    return any(char.isdigit() for char in inputString)


In [4]:
# Dict Parteien 

In [5]:
# Offizielle Listennummer mit Parteiname BTW 2021
# Offizielle Listennummer mit Parteiname BTW 2021
parteien_dict = {"1":"AfD", "2":"CDU", "3":"LINKE", "4":"SPD", "5":"FDP", "6":"GRÜNE", \
                # kleinere Parteien (sonstige)
                "7":"Tierschutzpartei", "8":"Die Partei", "9":"NPD", "10":"FREIE WÄHLER", \
                 "11":"PIRATEN", "12":"ÖDP", "13":"V-Partei³", "14":"MLPD", "15":"dieBasis", 
                 "16": "Bündnis C", "17": "III. Weg", "18":"DKP", "19":"Die Humanisten", \
                 "20":"Gesundheitsforschung", "21":"Team Todenhöfer", "22":"Volt", \
                 # kleinere Parteien ohne Landesliste
                 "23": "ASMUSfddB", "24":"AufrechterDemokrat", "25":"Brähmig", "26":"BüSo", \
                 "27":"Daniel Richter", "28":"Direktkandidat Daniel Zimmet", "29":"Direktkandidat Frank Hannig", \
                 "30": "DK Schöllner", "31":"Ein Leipziger für Leipzig", "32":"#KLIMAfirst", \
                 "33": "Internationalistisches Bündnis", "34": "Internationalistisches Bündnis", \
                 "35": "Internationalistisches Bündnis", "36":"Ischgl", "37":"KARLKOMMT", \
                 "38": "LKR", "39": "Marcus Fuchs - Querdenken 351", "40":"Queck", "41":"REICHEL",\
                 "42":"Röhder", "43":"UNO - Menschrecht - FRIEDEN für Alle", "44":"WIR sind LEIPZIGER", \
                 "45":"www.zeitkaufhaus.de Martin Bayer", \
                 # Bezeichner für gültige und ungültige Stimmen
                "GUE": "gültige Stimmen", "UNG": "Ungültige Stimmen"

                }

In [6]:


# jedes elem in root ist der Tag <Gebietsergebnis>, in dem immer eine komplette Gemeinde steckt. 
def read_xml(root):
    
    # Variablen in Datensatz
    erststimmen = "DIREKT"
    zweitstimmen = "LISTE"


    
    # zwei Dataframes werden erstellt, eins für die Ergebnisse der Parteien, 
    # ein anderes für Wahlstatistiken etc. 
    
    # DF Ergebnisse
    cols_result = ["KreisId", "KreisName", "Partei", "Stimmart", "Stimmen", "Anteil", "Stand"]
    df_results = pd.DataFrame(columns=cols_result)
    
    # DF Statistiken
    col_stats = ["KreisId", "KreisName","Statistik", "Stimmen", "Anteil", "Stand"]
    df_stats = pd.DataFrame(columns=col_stats)
    
    # DF Wahlkreis-Sieger
    col_winner = ["KreisId", "KreisName", "Sieger"]
    df_winner = pd.DataFrame(columns=col_winner)
    
    # Durchlauf durch alle <Gruppenergebnis>, sammeln der Gemeindedaten
    for elem in root:
        
        kreis_id = elem.attrib.get("KreisNr")
        stand = elem.attrib.get("Stand") 
        kreis_nm = elem.attrib.get("KreisName")
        
        # Unterpunkt ist <Gruppenergebnis>, hier findet sich unter "Position" die Beschreibung des jew. Datensatzes
        for sub in elem: 
            position = sub.attrib.get("Position")
            
            # Check ob position eine Nummer ist - dann handelt es sich nämlich um ein Partei-Stimmergebnis
            # dann wird das result-Dataframe befüllt
            if (has_numbers(position)) | (position == "UNG") | (position == "GUE"): 
                partei = parteien_dict[position]
                for node in sub: 
                    kind_of_votes = node.attrib.get("Stimmart")
                    if kind_of_votes == erststimmen: 
                        stimmart = "Erststimme"
                    else: 
                        stimmart = "Zweitstimme"
                    count_votes = node.attrib.get("Stimmen")
                    votes_share = node.attrib.get("Anteil")
                    ser_results = pd.Series([kreis_id, kreis_nm, partei, stimmart, count_votes, votes_share, stand], index=cols_result)
                    df_results = df_results.append(ser_results, ignore_index=True)
            
            # wenn Buchstaben-Kennung wird das statistische Dataframe befüllt
            
            elif position=="SIEGER": 
                info_node = sub[0]
                sieger = info_node.attrib.get("Sieger")
                ser_win = pd.Series([kreis_id,kreis_nm, sieger], index=col_winner)
                df_winner = df_winner.append(ser_win, ignore_index=True)
            
            else:
                
                info_node = sub[0]
                stimmen = info_node.attrib.get("Stimmen")
                anteil = info_node.attrib.get("Anteil")
                
                if position == "WB": 
                    statistik = "Wahlberechtigte"
                    anteil = "100"
                elif position == "WAE":
                    statistik = "Wähler"
              
                else:
                    print(position)
                    print("Fehler")
               
                ser_stats = pd.Series([kreis_id, kreis_nm, statistik, stimmen, anteil, stand], index=col_stats)
                df_stats = df_stats.append(ser_stats, ignore_index=True)
        
    return df_results, df_stats, df_winner
                    
                    
            #if position
            
           

            
            
            
df_ergebnisse, df_daten, df_sieger = read_xml(root)

In [7]:
df_ergebnisse

,KreisId,KreisName,Partei,Stimmart,Stimmen,Anteil,Stand
0,151,Nordsachsen,Ungültige Stimmen,Erststimme,1168,1.0,30 von 30 Gemeinden
1,151,Nordsachsen,Ungültige Stimmen,Zweitstimme,1205,1.0,30 von 30 Gemeinden
2,151,Nordsachsen,gültige Stimmen,Erststimme,114614,99.0,30 von 30 Gemeinden
3,151,Nordsachsen,gültige Stimmen,Zweitstimme,114577,99.0,30 von 30 Gemeinden
4,151,Nordsachsen,AfD,Erststimme,28561,24.9,30 von 30 Gemeinden
...,...,...,...,...,...,...,...
601,166,Vogtlandkreis,Gesundheitsforschung,Zweitstimme,0,0.0,37 von 37 Gemeinden
602,166,Vogtlandkreis,Team Todenhöfer,Zweitstimme,0,0.0,37 von 37 Gemeinden
603,166,Vogtlandkreis,Volt,Zweitstimme,0,0.0,37 von 37 Gemeinden
604,166,Vogtlandkreis,ASMUSfddB,Erststimme,116,0.1,37 von 37 Gemeinden


# Daten von long auf wide umstellen

In [8]:
df_data_wide = pd.pivot(df_daten, index=["KreisId", "KreisName"], columns="Statistik", values=["Stimmen", "Anteil"])\
    .reset_index()

In [9]:
# für die results müssen wir Erst- und Zweitstimmen vorher trennen, da es sonst zu Dopplungen kommt. 
df_ergebnisse_es = df_ergebnisse[df_ergebnisse["Stimmart"]=="Erststimme"]
df_ergebnisse_zs = df_ergebnisse[df_ergebnisse["Stimmart"]=="Zweitstimme"]
#del df_ergebnisse_es["Stimmart"]

In [10]:
df_ergebnisse_es

,KreisId,KreisName,Partei,Stimmart,Stimmen,Anteil,Stand
0,151,Nordsachsen,Ungültige Stimmen,Erststimme,1168,1.0,30 von 30 Gemeinden
2,151,Nordsachsen,gültige Stimmen,Erststimme,114614,99.0,30 von 30 Gemeinden
4,151,Nordsachsen,AfD,Erststimme,28561,24.9,30 von 30 Gemeinden
6,151,Nordsachsen,CDU,Erststimme,14380,12.5,30 von 30 Gemeinden
8,151,Nordsachsen,LINKE,Erststimme,14389,12.6,30 von 30 Gemeinden
...,...,...,...,...,...,...,...
590,166,Vogtlandkreis,ÖDP,Erststimme,14948,11.0,37 von 37 Gemeinden
594,166,Vogtlandkreis,dieBasis,Erststimme,5687,4.2,37 von 37 Gemeinden
597,166,Vogtlandkreis,III. Weg,Erststimme,694,0.5,37 von 37 Gemeinden
604,166,Vogtlandkreis,ASMUSfddB,Erststimme,116,0.1,37 von 37 Gemeinden


In [11]:
df_results_es_wide = pd.pivot(df_ergebnisse_es, index=["KreisId", "KreisName"], columns="Partei", values=["Stimmen", "Anteil"])\
    .reset_index().fillna("0")
df_results_zs_wide = pd.pivot(df_ergebnisse_es, index=["KreisId", "KreisName"], columns="Partei", values=["Stimmen", "Anteil"])\
    .reset_index().fillna("0")

In [12]:
df_results_es_wide.columns = [" ".join(col).strip() for col in df_results_es_wide.columns.values]
df_results_zs_wide.columns = [" ".join(col).strip() for col in df_results_zs_wide.columns.values]

In [13]:
# Suffix _1 für Erststimme, _2 für Zweitstimme hinzufügen
df_results_es_wide = df_results_es_wide.add_prefix("1_")
df_results_zs_wide = df_results_zs_wide.add_prefix("2_")

# Columns AgS und Ort Prefix entfernen
df_results_es_wide.rename(columns={"1_KreisId":"KreisId", "1_KreisName":"KreisName"}, inplace=True)
df_results_zs_wide.rename(columns={"2_KreisId":"KreisId", "2_KreisName":"KreisName"}, inplace=True)

,KreisId,KreisName,2_Stimmen ASMUSfddB,2_Stimmen AfD,2_Stimmen AufrechterDemokrat,2_Stimmen Brähmig,2_Stimmen BüSo,2_Stimmen Bündnis C,2_Stimmen CDU,2_Stimmen Die Humanisten,...,2_Anteil III. Weg,2_Anteil LINKE,2_Anteil MLPD,2_Anteil PIRATEN,2_Anteil SPD,2_Anteil Ungültige Stimmen,2_Anteil V-Partei³,2_Anteil dieBasis,2_Anteil gültige Stimmen,2_Anteil ÖDP
0,151,Nordsachsen,85,28561,0,0,0,0,14380,0,...,0,12.6,0,0,25.0,1.0,0,0.2,99.0,1.0
1,152,Leipzig I,990,21716,1045,1035,1029,0,32840,986,...,0,12.7,0.7,0,13.7,3.1,0,0.7,96.9,0.7
2,153,Leipzig II,1031,23658,0,0,0,0,33775,1051,...,0,12.7,0.7,0.7,13.8,3.0,0,0.7,97.0,0.7
3,154,Leipzig-Land,624,37784,0,0,0,0,19932,0,...,0,12.6,0,0,24.1,1.1,0,0.2,98.9,0.9
4,155,Meißen,4923,32291,0,0,0,0,17171,0,...,0,11.8,0,0,22.2,1.4,0,1.2,98.6,1.9
5,156,Bautzen I,566,35125,0,0,0,0,17545,0,...,0,12.5,0,0,25.0,1.0,0,3.3,99.0,0
6,157,Görlitz,5181,37002,1285,0,0,0,18460,0,...,0,12.5,0,0,25.0,1.0,0,10.2,99.0,0
7,158,Sächsische Schweiz-Osterzgebirge,212,35191,39,0,0,0,17562,0,...,0,12.4,0,0,24.9,1.0,0,0.9,99.0,3.7
8,159,Dresden I,0,35343,0,0,0,3,17569,0,...,0,12.5,6.8,1.1,24.9,1.0,0,0.4,99.0,0.8
9,160,Dresden II - Bautzen II,1,37320,22,0,0,0,18676,0,...,0,12.5,0.1,1.4,24.9,1.0,0,3.2,99.0,1.0


In [14]:
# komplette Result-Tabelle anlegen
df_results_merge = pd.merge(df_results_es_wide, df_results_zs_wide, on=["KreisId", "KreisName"])

In [15]:
# Results mit Statistik mergen
df_all_merge = pd.merge(df_results_merge, df_data_wide, on=["KreisId", "KreisName"])

<ipython-input-15-966e9356011c>:2: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)
  df_all_merge = pd.merge(df_results_merge, df_data_wide, on=["KreisId", "KreisName"])
c:\users\c_knoop\appdata\local\programs\python\python39\lib\site-packages\pandas\core\generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


## Gewinner 1. und 2.STimmen festlegen


In [16]:
# richtige Formate vergeben
df_all_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16 entries, 0 to 15
Data columns (total 98 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   KreisId                         16 non-null     object
 1   KreisName                       16 non-null     object
 2   1_Stimmen ASMUSfddB             16 non-null     object
 3   1_Stimmen AfD                   16 non-null     object
 4   1_Stimmen AufrechterDemokrat    16 non-null     object
 5   1_Stimmen Brähmig               16 non-null     object
 6   1_Stimmen BüSo                  16 non-null     object
 7   1_Stimmen Bündnis C             16 non-null     object
 8   1_Stimmen CDU                   16 non-null     object
 9   1_Stimmen Die Humanisten        16 non-null     object
 10  1_Stimmen Die Partei            16 non-null     object
 11  1_Stimmen FDP                   16 non-null     object
 12  1_Stimmen FREIE WÄHLER          16 non-null     obje

In [17]:
df_all_merge["2_Gewinner"] = df_all_merge[['2_Stimmen AfD', '2_Stimmen CDU',
       '2_Stimmen FDP', '2_Stimmen GRÜNE', '2_Stimmen LINKE', '2_Stimmen SPD']].idxmax(axis=1)
df_all_merge["2_Gewinner"] = df_all_merge["2_Gewinner"].str.split(" ")
df_all_merge["2_Gewinner"] = df_all_merge["2_Gewinner"].map(lambda x: x[1])

df_all_merge["1_Gewinner"] = df_all_merge[['1_Stimmen AfD', '1_Stimmen CDU',
       '1_Stimmen FDP', '1_Stimmen GRÜNE', '1_Stimmen LINKE', '1_Stimmen SPD']].idxmax(axis=1)
df_all_merge["1_Gewinner"] = df_all_merge["1_Gewinner"].str.split(" ")
df_all_merge["1_Gewinner"] = df_all_merge["1_Gewinner"].map(lambda x: x[1])


TypeError: reduction operation 'argmax' not allowed for this dtype

In [242]:
df_all_merge.to_csv("wahl_testdaten.csv", index=False)